In [23]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV, LinearRegression
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

import numpy as np

# Carregar e visualizar os dados

In [24]:
# Carrear os dados - Regressão
df = pd.read_csv('datasets/dataset_colesterol.csv')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     996 non-null    object 
 2   Fumante             997 non-null    object 
 3   Nível de Atividade  996 non-null    object 
 4   Idade               997 non-null    float64
 5   Peso                997 non-null    float64
 6   Altura              997 non-null    float64
 7   Colesterol          1000 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 62.6+ KB


In [26]:
# Ajustar o df
df.drop('Id', axis=1, inplace=True)

# Coletar medidas das variaveis categoricas
moda_grupo_sanguineo = df['Grupo Sanguíneo'].mode()
moda_fumante = df['Fumante'].mode()
moda_nivel_atividade_fisica = df['Nível de Atividade'].mode()

# Coletar medidas das variaveis numericas
media_idade = df['Idade'].mean()
media_peso = df['Peso'].mean()   
media_altura = df['Altura'].mean()

# Tratar valores ausentes
df.fillna(value={
    'Grupo Sanguíneo': moda_grupo_sanguineo[0],
    'Fumante': moda_fumante[0],
    'Nível de Atividade': moda_nivel_atividade_fisica[0],
    'Idade': media_idade,
    'Peso': media_peso,
    'Altura': media_altura    
}, inplace=True)

# Aplicar One Hot Encoding nas variáveis categóricas
df = pd.get_dummies(df, columns=["Grupo Sanguíneo", "Fumante", "Nível de Atividade"])

In [27]:
# DF atualizado
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Idade                        1000 non-null   float64
 1   Peso                         1000 non-null   float64
 2   Altura                       1000 non-null   float64
 3   Colesterol                   1000 non-null   float64
 4   Grupo Sanguíneo_A            1000 non-null   uint8  
 5   Grupo Sanguíneo_AB           1000 non-null   uint8  
 6   Grupo Sanguíneo_B            1000 non-null   uint8  
 7   Grupo Sanguíneo_O            1000 non-null   uint8  
 8   Fumante_Não                  1000 non-null   uint8  
 9   Fumante_Sim                  1000 non-null   uint8  
 10  Nível de Atividade_Alto      1000 non-null   uint8  
 11  Nível de Atividade_Baixo     1000 non-null   uint8  
 12  Nível de Atividade_Moderado  1000 non-null   uint8  
dtypes: float64(4), uint

# Treinar modelo de regressão com Lasso (L1)

In [28]:
# Separar X e Y
X = df.drop('Colesterol', axis=1)
y = df['Colesterol']

In [29]:
# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [30]:
# Treinar o modelo de regressão linear múltipla com regularização Lasso
# Quanto maior o alpha, maior a penalização e mais coefientes serão zerados
model_lasso = Lasso(alpha=0.1)
model_lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [31]:
# Mostrar importância das features
def importancia_features(model: LinearRegression):
    importance = np.abs(model.coef_)
    print("Importância das features:")
    for i, feature in enumerate(model.feature_names_in_):
        print(f"{feature}: {importance[i]}")

In [32]:
# Importância das features
importancia_features(model_lasso)

Importância das features:
Idade: 0.02119914334658416
Peso: 2.482098417216475
Altura: 2.194647428228912
Grupo Sanguíneo_A: 0.1181197780671792
Grupo Sanguíneo_AB: 1.0931753184051953
Grupo Sanguíneo_B: 0.5903206465632868
Grupo Sanguíneo_O: 0.023748076938104875
Fumante_Não: 1.6895280124754466
Fumante_Sim: 0.0
Nível de Atividade_Alto: 1.9226345989142501
Nível de Atividade_Baixo: 0.0
Nível de Atividade_Moderado: 0.0


In [33]:
# Mostrar a perfornance do modelo
def mostrar_performance(modelo, X_test, y_test):
    # Fazer previsões
    y_pred = modelo.predict(X_test)

    # Calcular o erro
    return root_mean_squared_error(y_test, y_pred)

In [35]:
# Mostrar performance
print(f"Lasso: {mostrar_performance(model_lasso, X_test, y_test)}")

Lasso: 8.979177521813472
